In [1]:
from pyReef.model import Model 

In [2]:
# initialise model
reef = Model()

In [3]:
# Define the XmL input file
reef.load_xml('input.xml')

Swan model initialisation took 0.09 seconds


In [ ]:
print reef.pyStrat.stratPerc[10,10,0,:]

In [ ]:
print reef.pyGrid.regZ.shape,reef.pyGrid.partIDs
print reef.force.waveNb,reef.force.waveWu

In [ ]:
%matplotlib inline

import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt

# display plots in SVG format
%config InlineBackend.figure_format = 'svg' 

plt.figure(figsize = (5,5))
ax = plt.gca()

# Map
im = ax.imshow(reef.pyGrid.regZ.T, extent=(0,1,0,1), origin='lower', aspect=1)
im.set_cmap('terrain')

# Colorbar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="10%", pad=0.1, aspect=20)
cb = plt.colorbar(im, cax=cax)
cb.set_label('topography',size=12)

plt.show()

In [ ]:
!mpiexec -n 1 python mpi_example.py 10000

In [ ]:
reef.pyGrid.regZ.shape

In [ ]:
reef.pyGrid.regX.shape

In [ ]:
reef.pyGrid.regY.shape

In [ ]:
x,y = np.meshgrid(reef.pyGrid.regX, reef.pyGrid.regY)

In [ ]:
x.shape

In [ ]:
!mpiexec -n 2 python mpi_example.py 10000

In [ ]:
!mpiexec -n 1 python mpi_example.py 10000

In [ ]:
!mpiexec -n 2 python mpi_example.py 10000

In [ ]:
!mpiexec -n 3 python mpi_example.py 10000